# Homework: Phrasal Chunking

This is your documentation for the chunker homework.

In [1]:
import numpy as np 
import pandas as pd 
import itertools 
import smtplib
import perc
import default
import sys
from collections import defaultdict
import gzip # use compressed data files
import copy, operator, optparse, sys, os
import re 
import time

In [22]:
feat_vec = {}
tagset = []
train_data = []

tagset = perc.read_tagset("data/tagset.txt")
train_data = perc.read_labeled_data("data/train.txt.gz", "data/train.feats.gz", verbose=False)
print("reading data ...", file=sys.stderr)

reading data ...


In [29]:
def perc_train(train_data, tagset, numepochs):
    t1=time.time()
    FEATURE_VEC_SENT=[] 
    FEATURE_DIC={}
    default_tag=tagset[0]
    WEIGHT_VECTOR=[]
    e=0
    for epoch in range(numepochs) : 
        print("epoch {}".format(e))
        
        sent_ind=0
        ## ONE SENTENCE PARSING 
        
        for (sent_labels,sent_features) in train_data : 
            
            #print("sentence=",sent_labels)
            FEATURE_DIC={}
            
            argmax_tag=[]
             
            if epoch==0 : 
                FEATURE_DIC={}
            elif epoch>0: 
                FEATURE_DIC=WEIGHT_VECTOR[sent_ind]

                
            argmax_tag=perc.perc_test(FEATURE_DIC, sent_labels, sent_features, tagset, default_tag)   
            
            labels = copy.deepcopy(sent_labels)
            labels.insert(0, '_B-1 _B-1 _B-1')
            labels.insert(0, '_B-2 _B-2 _B-2') # first two 'words' are _B-2 _B-1
            labels.append('_B+1 _B+1 _B+1')
            labels.append('_B+2 _B+2 _B+2') # last two 'words' are _B+1 _B+2  
            N = len(labels)   
            
            SUM=0
            ## for each word chunk
            feat_index=0
            j=0
            for i in range(2, N-2):
                
                (feat_index, feats) = perc.feats_for_word(feat_index, sent_features)  
               # print(feats)
                fields = labels[i].split()    
                
                
                word = fields[0]
                actual_tag=fields[2]    
            
                
                #print("word_index = {}, word = {}, actual_tag = {}, argmax_tag= {}".format(i,word,actual_tag,argmax_tag[i])
                amax_tag=argmax_tag[j]
                
                local_keys=list(FEATURE_DIC.keys())
                feat_list=[(x,actual_tag,amax_tag) for x in feats]
                #print(feat_list)
                
                
                ## UPDATE THE WEIGHTS HERE.
                for (f,b,c) in feat_list :  
                    
                    if b!=c : 
                        SUM+=1
                        if (f,b) in local_keys : 
                            FEATURE_DIC[(f,b)]=FEATURE_DIC[(f,b)]+1
                        else :
                            FEATURE_DIC[(f,b)]=1
                    
                        if (f,c) in local_keys : 
                            FEATURE_DIC[(f,c)]=FEATURE_DIC[(f,c)]-1
                        else : 
                            FEATURE_DIC[(f,c)]=-1
                            
                j=j+1                 
            
            
           
            
            if epoch == 0 : 
                WEIGHT_VECTOR.append(dict(list(FEATURE_DIC.items())))   
            else :
                WEIGHT_VECTOR[sent_ind]=dict(list(FEATURE_DIC.items()))
            sent_ind=sent_ind+1  
            
            
        e=e+1
        
        
    print("Training finished in ",time.time()-t1)
    return WEIGHT_VECTOR
                
            
            
feat_vec = perc_train(train_data, tagset, 2)
perc.perc_write_to_file(feat_vec, "default.model")
print("wrote model to disk")

epoch 0
epoch 1
Training finished in  279.10464882850647
wrote model to disk


In [33]:
F={}
for i in range(len(feat_vec)) : 
    a=feat_vec[i].copy()
    F.update(a)
feat_vec=F.copy()

In [ ]:
%%capture --no-stderr output
print("reading test data ...", file=sys.stderr)
test_data = perc.read_labeled_data("data/dev.txt", "data/dev.feats", verbose=False)
print("done.", file=sys.stderr)
feat_vec = perc.perc_read_from_file("default.model")
perc.perc_testall(feat_vec, test_data, tagset)

reading test data ...
done.


In [27]:
import score_chunks
boundary = "-X-" # something to use as boundary between sentences
outside = "O" # tag used to mark the outside of any chunk
conlleval = False # use conlleval (should be False for most use cases)
numfeats = 2 # number of columns to consider as features, typically "word POStag"
(test, _) = score_chunks.readTestFile(str(output), boundary, outside, conlleval, numfeats)
with open("data/reference500.txt") as f:
    (reference, _) = score_chunks.readTestFile(f.read(), boundary, outside, conlleval, numfeats)
print("Score: %.2f" % score_chunks.corpus_fmeasure(reference, test, False))

processed 500 sentences with 10375 tokens and 5783 phrases; found phrases: 5983; correct phrases: 1314
             ADJP: precision:   0.00%; recall:   0.00%; F1:   0.00; found:      0; correct:     99
             ADVP: precision:   5.37%; recall:  11.39%; F1:   7.30; found:    428; correct:    202
            CONJP: precision:   0.00%; recall:   0.00%; F1:   0.00; found:      0; correct:      5
             INTJ: precision:   0.00%; recall:   0.00%; F1:   0.00; found:      0; correct:      1
               NP: precision:   7.92%; recall:   2.71%; F1:   4.04; found:   1036; correct:   3026
               PP: precision:  44.76%; recall:  77.64%; F1:  56.78; found:   2118; correct:   1221
              PRT: precision:   0.00%; recall:   0.00%; F1:   0.00; found:      0; correct:     22
             SBAR: precision:   0.00%; recall:   0.00%; F1:   0.00; found:      0; correct:    107
               VP: precision:  10.87%; recall:  23.73%; F1:  14.91; found:   2401; correct:   1100
accura

In [28]:
feat_vec

{('U00:_B-1', 'B-VP'): 1,
 ('U00:_B-1', 'B-NP'): -1,
 ('U01:It', 'B-VP'): 1,
 ('U01:It', 'B-NP'): -1,
 ('U02:is', 'B-VP'): 1,
 ('U02:is', 'B-NP'): -1,
 ('U03:also', 'B-VP'): 1,
 ('U03:also', 'B-NP'): -1,
 ('U04:pulling', 'B-VP'): 1,
 ('U04:pulling', 'B-NP'): -1,
 ('U05:It/is', 'B-VP'): 1,
 ('U05:It/is', 'B-NP'): -1,
 ('U06:is/also', 'B-VP'): 1,
 ('U06:is/also', 'B-NP'): -1,
 ('U10:_B-1', 'B-VP'): 1,
 ('U10:_B-1', 'B-NP'): -1,
 ('U11:PRP', 'B-VP'): 1,
 ('U11:PRP', 'B-NP'): -2,
 ('U12:VBZq', 'B-VP'): 1,
 ('U12:VBZq', 'B-NP'): -1,
 ('U13:RB', 'B-VP'): 1,
 ('U13:RB', 'B-NP'): -2,
 ('U14:VBG', 'B-VP'): 1,
 ('U14:VBG', 'B-NP'): -1,
 ('U15:_B-1/PRP', 'B-VP'): 1,
 ('U15:_B-1/PRP', 'B-NP'): -1,
 ('U16:PRP/VBZ', 'B-VP'): 1,
 ('U16:PRP/VBZ', 'B-NP'): -1,
 ('U17:VBZ/RB', 'B-VP'): 1,
 ('U17:VBZ/RB', 'B-NP'): -1,
 ('U18:RB/VBG', 'B-VP'): 1,
 ('U18:RB/VBG', 'B-NP'): -1,
 ('U20:_B-1/PRP/VBZ', 'B-VP'): 1,
 ('U20:_B-1/PRP/VBZ', 'B-NP'): -1,
 ('U21:PRP/VBZ/RB', 'B-VP'): 1,
 ('U21:PRP/VBZ/RB', 'B-NP'): -1